# BBC - Projet
## Diagnostic miARN pour détecter le cancer de la prostate

- Professeur: Carlos Peña (<a href="mailto:carlos.pena@heig-vd.ch">carlos.pena@heig-vd.ch</a>)
- Assistante: Aitana Lebrand (<a href="mailto:aitana.nevesdasilva@heig-vd.ch">aitana.nevesdasilva@heig-vd.ch</a>)
- Etudiant: Jan Purro (<a href="mailto:jan.purro@heig-vd.ch">jan.purro@heig-vd.ch</a>)
- Etudiant: Thibault Schowing (<a href="mailto:thibault.schowing@heig-vd.ch">thibault.schowing@heig-vd.ch</a>)

Date: Printemps 2016

> ## Objectifs pédagogiques


> ## But du projet



> ## Démarche


> ## Rapport de labo



In [12]:
import os
import numpy as np
import pylab as plt
import pandas as pd
import numpy as np
from Bio import Geo

# Fonctionne pour les fichiers .soft
def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records

# Les différents records, le soft n'es pas forcément utile.

#records_soft = load_geo('../data/GSE45604_family.soft')
records_matrix = open('../data/GSE45604_series_matrix.txt')

# Données brutes (uniquement les lignes avec condition et miARN)
matrice =  [l.split('\t') for l in records_matrix.readlines()[67:-1]]
records_matrix.seek(0)
conditions = records_matrix.readlines()[34].split('\t')


# matrice <list> -> <ndarray>
matrice = np.array(matrice)
conditions = np.array(conditions[1:])

# On a 2 types de cellules: prostate normale ou cancéreuse qu'on va marquer PCa ou Normal
idx_normal = []
idx_pca = []
i = 1
for c in conditions:
    if c.startswith('"PCa'):
        matrice[0,i] = 'PCa'
        idx_pca.append(i)
    else:
        matrice[0,i] = 'Normal'
        idx_normal.append(i)
    i += 1

# TODO: mettre dans l'ordre Normale puis PCa
idx_ordered = [0] + idx_normal + idx_pca
print("idx: ")
print(idx_ordered)      

# AVANT TRANSPOSITION
# Matrice[0] contient ID_REF - PCa - PCa - ... - Normal - PCa - ...
# Matrice[1] contient 234342_st - 23.3 - 342.3 - ...
# Matrice[:,0] contient ID_REF suivi de tous les id de gènes
# Matrice[:,1...n] Contient PCa ou normal suivi de toutes les mesures pour ce prélèvement sur un patient atteint ou non
print matrice

# On réordonne avec les Idx dans le bon ordre
matrice = matrice[:,idx_ordered]

idx: 
[0, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]
[['"ID_REF"' 'PCa' 'PCa' ..., 'PCa' 'PCa' 'PCa']
 ['"1367452_st"' '10.93488' '13.60777' ..., '16.10932' '14.50111'
  '15.29177\n']
 ['"1367453_st"' '14.42092' '15.43682' ..., '15.15539' '17.9957'
  '25.17342\n']
 ..., 
 ['"zma-miR529_st"' '15.90142' '20.75771' ..., '12.53141' '13.16191'
  '13.03476\n']
 ['"zma-miR827-star_st"' '14.19222' '17.82426' ..., '14.37837' '16.76542'
  '16.02319\n']
 ['"zma-miR827_st"' '15.00203' '13.44333' ..., '14.51185' '11.54497'
  '13.00997\n']]


In [15]:
# Diviser le dataset en deux -> test et train


print matrice[0,:]

#print(np.shape(matrice)) # (20644,61)

idx_test = [5,9,19,28,32,60]
idx_train = range(0,len(matrice[0]))
[idx_train.remove(i) for i in idx_test]
idx_train.remove(0)

print("IDX test: ", idx_test)
print("IDX train: ", idx_train)

data = matrice
data_test = data[:,idx_test]
data = data[:,idx_train]

print("Matrice Test", matrice[:, idx_test])


['"ID_REF"' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal'
 'Normal' 'Normal' 'Normal' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa'
 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa'
 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa'
 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa'
 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa']
('IDX test: ', [5, 9, 19, 28, 32, 60])
('IDX train: ', [1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])
('Matrice Test', array([['Normal', 'Normal', 'PCa', 'PCa', 'PCa', 'PCa'],
       ['11.47406', '13.86868', '14.12206', '14.15627', '11.64925',
        '15.29177\n'],
       ['14.39909', '16.66849', '16.31204', '20.05932', '15.40274',
        '25.17342\n'],
       ..., 
       ['15.43849', '17.41694', '14.67161', '16.80459', '

In [16]:
# On va mettre les datas dans l'ordre

#print(list(matrice[0,1:]))
print(data)




[['Normal' 'Normal' 'Normal' ..., 'PCa' 'PCa' 'PCa']
 ['17.73092' '13.50618' '15.66553' ..., '17.9872' '16.10932' '14.50111']
 ['15.98614' '17.80959' '16.43776' ..., '19.47316' '15.15539' '17.9957']
 ..., 
 ['18.99103' '13.52609' '13.41379' ..., '13.46852' '12.53141' '13.16191']
 ['15.40228' '14.92211' '16.40466' ..., '15.38391' '14.37837' '16.76542']
 ['17.98331' '17.41605' '14.81159' ..., '10.7645' '14.51185' '11.54497']]
